---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',None)

In [4]:
def get_list_of_university_towns():    
   
    with open('university_towns.txt',encoding='utf-8') as file:
        lst = []
        state = ''
        region = ''
        st_region = []
        cnt = 0
        for line in file:
            if '[ed' in line:
                state = line.split('[ed')[0].strip()
            elif '(' in line:
                cnt = cnt + 1
                region = line.split('(')[0].strip()
                st_region = [state,region]
                lst.append(st_region)
            else:
                region = line.strip('\n')
                st_region = [state,region]
                lst.append(st_region)

    df = pd.DataFrame(data=lst,columns=['State','RegionName'])
    return df
#get_list_of_university_towns()

In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls')
    df = df.drop(df.index[0:7])
    df = df.drop('Unnamed: 3',axis=1)
    df = df.drop(df.columns[[-1]],axis=1)
    df = df.reset_index(drop=True)
    df = df.rename_axis({'Unnamed: 1':'Annual - GDP in billions of current dollars',
                            'Unnamed: 2':'Annual - GDP in billions of chained 2009 dollars',
                            'Unnamed: 4':'GDP Quarter',
                            'Unnamed: 5':'GDP in billions of current dollars',
                            'Unnamed: 6':'GDP in billions of chained 2009 dollars'}, axis=1)
    df = df.fillna('NA')
    df = df.iloc[210:]
    df = df.reset_index(drop=True)
    df_new = df[['GDP Quarter','GDP in billions of chained 2009 dollars']]
    df_new['diff'] = df['GDP in billions of chained 2009 dollars']- df['GDP in billions of chained 2009 dollars'].shift(1)
    df_new = df_new.drop('GDP in billions of chained 2009 dollars', axis = 1)
    for index,row in df_new.iterrows():
        if index<len(df_new)-2:
            val1 = df_new.iloc[index][1]
            val2 = df_new.iloc[index+1][1]
            if (val1<0) & (val2<0):
                break
    return df_new.iloc[index][0]


In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls')
    df = df.drop(df.index[0:7])
    df = df.drop('Unnamed: 3',axis=1)
    df = df.drop(df.columns[[-1]],axis=1)
    df = df.reset_index(drop=True)
    df = df.rename_axis({'Unnamed: 1':'Annual - GDP in billions of current dollars',
                            'Unnamed: 2':'Annual - GDP in billions of chained 2009 dollars',
                            'Unnamed: 4':'GDP Quarter',
                            'Unnamed: 5':'GDP in billions of current dollars',
                            'Unnamed: 6':'GDP in billions of chained 2009 dollars'}, axis=1)
    df = df.fillna('NA')
    df = df.iloc[210:]
    df = df.reset_index(drop=True)
    df_new = df[['GDP Quarter','GDP in billions of chained 2009 dollars']]
    df_new['diff'] = df['GDP in billions of chained 2009 dollars']- df['GDP in billions of chained 2009 dollars'].shift(1)
    df_new = df_new.drop('GDP in billions of chained 2009 dollars', axis = 1)
    start = ''
    for index,row in df_new.iterrows():
        if index<len(df_new)-2:
            val1 = df_new.iloc[index][1]
            val2 = df_new.iloc[index+1][1]
            if (val1<0) & (val2<0):
                break
    start = index
    end = ''
    for index,row in df_new.iloc[start:].iterrows():
        if index<len(df_new)-2:
            val1 = df_new.iloc[index][1]
            val2 = df_new.iloc[index+1][1]
            if (val1>0) & (val2>0):
                break
    end = index+1
    return df_new.iloc[end][0]


In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls')
    df = df.drop(df.index[0:7])
    df = df.drop('Unnamed: 3',axis=1)
    df = df.drop(df.columns[[-1]],axis=1)
    df = df.reset_index(drop=True)
    df = df.rename_axis({'Unnamed: 1':'Annual - GDP in billions of current dollars',
                            'Unnamed: 2':'Annual - GDP in billions of chained 2009 dollars',
                            'Unnamed: 4':'GDP Quarter',
                            'Unnamed: 5':'GDP in billions of current dollars',
                            'Unnamed: 6':'GDP in billions of chained 2009 dollars'}, axis=1)
    df = df.fillna('NA')
    df = df.iloc[210:]
    df = df.reset_index(drop=True)
    df_new = df[['GDP Quarter','GDP in billions of chained 2009 dollars']]
    df_new['diff'] = df['GDP in billions of chained 2009 dollars']- df['GDP in billions of chained 2009 dollars'].shift(1)
    df_new = df_new.drop('GDP in billions of chained 2009 dollars', axis = 1)
    start = ''
    for index,row in df_new.iterrows():
        if index<len(df_new)-2:
            val1 = df_new.iloc[index][1]
            val2 = df_new.iloc[index+1][1]
            if (val1<0) & (val2<0):
                break
    start = index
    end = ''
    for index,row in df_new.iloc[start:].iterrows():
        if index<len(df_new)-2:
            val1 = df_new.iloc[index][1]
            val2 = df_new.iloc[index+1][1]
            if (val1<0) & (val2>0):
                break
    end = index
    return df_new.iloc[end][0]

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df['State'] = df['State'].apply(lambda st:states.get(st))
    df = df.set_index(['State','RegionName'])
    df = df.loc[:,'2000-01':'2016-08']
    df.columns = pd.to_datetime(df.columns)
    mdf = df.resample('Q',axis=1).mean()
    mdf.columns = ['{}q{}'.format(x.year,x.quarter) for x in mdf.columns]
    return mdf
#convert_housing_data_to_quarters()

    

In [9]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    house_price = convert_housing_data_to_quarters()
    rec_start_q = get_recession_start()
    rec_start_before_qrt_idx = int(rec_start_q[len(rec_start_q)-1:len(rec_start_q)]) - 1
    last_char = rec_start_q[len(rec_start_q)-1:len(rec_start_q)]
    rec_start_q = rec_start_q.replace(last_char,str(rec_start_before_qrt_idx))

    df = house_price.loc[:,rec_start_q:get_recession_bottom()]
    df['price_ratio'] = df[rec_start_q].div(df[get_recession_bottom()])
    
    df_univ_towns = get_list_of_university_towns()
    df_univ_towns = df_univ_towns.set_index(['State','RegionName'])
    tuples =[tuple(x) for x in df_univ_towns.to_records()]
    univ_town = df.loc[df.index.isin(tuples)][['price_ratio']]
    non_univ_towns = df.loc[~df.index.isin(tuples)][['price_ratio']]
    value, pvalue = ttest_ind(univ_town,non_univ_towns,nan_policy='omit')
    #print(float(pvalue.data))
    different = True
    if pvalue < 0.01:
        different = True
    else:
        different = False
    mean_ratio_town = ''
    if univ_town['price_ratio'].mean() < non_univ_towns['price_ratio'].mean():
        mean_ratio_town = "university town"
    else:
        mean_ratio_town = "non-university town"
    return (different,float(pvalue.data),mean_ratio_town)
(run_ttest())

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True, 0.002724063704753125, 'university town')